In [2]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

# 回测框架
import backtrader as bt

# 基础函数
import utilsJ

# 策略

In [12]:
class Strategy_Bollinger1(bt.Strategy):
    
    params = (
        ('printlog', False),
        ('bollinger_per', 20),
        ('bollinger_dev', 2),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.buyprice = None
        self.sellprice = None
        
        # Add Bollinger Bands indicator
        self.lines.top = bt.indicators.BollingerBands(self.datas[0], 
                                                      period = self.params.bollinger_per, 
                                                      devfactor = self.params.bollinger_dev).top
        self.lines.bot = bt.indicators.BollingerBands(self.datas[0], 
                                                      period = self.params.bollinger_per, 
                                                      devfactor = self.params.bollinger_dev).bot

        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                
                self.log('BUY EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                         (order.executed.price,
                          order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))
                self.buyprice = order.executed.price

            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                        (order.executed.price,
                          -order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))
                self.sellprice = order.executed.price

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
    def next(self):
        
        # Bollinger Buy Signal
        if (self.dataclose[0] > self.lines.top[0] and self.dataclose[-1] < self.lines.top[-1]) or (self.dataclose[0] > self.lines.bot[0] and self.dataclose[-1] < self.lines.bot[-1]):
            self.log('BUY CREATE, Price: %.2f, Lots: %i, Current Position: %i' % (self.dataclose[0], 
                                                                                         100, self.getposition(self.data).size))
            self.buy(size = 100)
        ### Bollinger Sell Singal
        elif  (self.dataclose[0] < self.lines.top[0] and self.dataclose[-1] > self.lines.top[-1]) or (self.dataclose[0] < self.lines.bot[0] and self.dataclose[-1] > self.lines.bot[-1]):
            if self.getposition(self.data).size > 0:
                    self.log('SELL CREATE (Close), Price: %.2f, Lots: %i' % (self.dataclose[0], 
                                                                                            self.getposition(self.data).size))
                    self.close()

# 回测

In [13]:
if __name__ ==  '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(Strategy_Bollinger1, printlog = True) 

    s_date = datetime.datetime.now() - datetime.timedelta(days = 365)
    e_date = datetime.datetime.now()

    # Create stock Data Feed
    stock_index = '002057.SZ'
    df = utilsJ.stock_tushare('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6', 
                        stock_index, s_date, e_date)
    data = bt.feeds.PandasData(dataname=df,fromdate=s_date,todate=e_date)
    
    # Add the index Data Feed to Cerebo
    cerebro.adddata(data)

    # Single stock testing 
#    df = get_data_ts('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6', 
#                        '000001.SZ', s_date, e_date)
#    data = bt.feeds.PandasData(dataname=df,fromdate=s_date,todate=e_date)
#    cerebro.adddata(data, name = '600600.SH')
    
    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(10000)

    # Set leverage
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
2021-10-29: BUY CREATE, Price: 7.80, Lots: 100, Current Position: 0
2021-11-01: BUY EXECUTED, Price: 7.80, Lot:100, Cash: 9220, Value: 10038
2021-11-03: SELL CREATE (Close), Price: 7.79, Lots: 100
2021-11-04: SELL EXECUTED, Price: 7.79, Lot:100, Cash: 9999, Value: 9999
2021-11-04: OPERATION PROFIT, GROSS -1.00, NET -1.00
2021-11-29: BUY CREATE, Price: 9.47, Lots: 100, Current Position: 0
2021-11-30: BUY EXECUTED, Price: 9.47, Lot:100, Cash: 9052, Value: 9967
2021-11-30: SELL CREATE (Close), Price: 9.15, Lots: 100
2021-12-01: SELL EXECUTED, Price: 9.15, Lot:100, Cash: 9967, Value: 9967
2021-12-01: OPERATION PROFIT, GROSS -32.00, NET -32.00
2022-01-28: BUY CREATE, Price: 8.01, Lots: 100, Current Position: 0
2022-02-07: BUY EXECUTED, Price: 8.01, Lot:100, Cash: 9166, Value: 9979
2022-02-28: BUY CREATE, Price: 9.44, Lots: 100, Current Position: 100
2022-03-01: BUY EXECUTED, Price: 9.44, Lot:100, Cash: 8222, Value: 10086
2022-03-01: SELL CREATE (Close), Pr